# 目标
爬取苦瓜书盘的数据，并下载图书文件。
这是一个单线程的爬虫，速度有些慢。

In [6]:
import os
from requests_html import HTMLSession
import urllib
import urllib.request
import requests
# https
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

In [7]:
session = HTMLSession()
# 创建一个ua，绕过初级反爬
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0'
headers = {'User-Agent': ua}

# 跳过前几页
firstPage = 27

In [8]:
# 获取页面最后一页的地址,str
def get_last_page_url():
    # 返回一个 response 对象
    response = session.get('https://kgbook.com/list/index.html', headers=headers)
    pages = response.html.find('div.pagenavi', first=True)
    a_list = pages.find('a')
    urlObject = a_list[len(a_list) - 1]
    u = urlObject.absolute_links
    ustr = ''.join(u)
    return ustr;

# 获取页的数量
def get_total_pages(ustr):
    back = ustr.split('index_')[1]
    num = back.split('.')[0]
    print("一共%s页" % num)
    return int(num);
    
# 创建页地址数组
def get_total_page_urls(num):
    urls = []
    for index in range(num):
        if index <= 0:
            urls.append('https://kgbook.com/list/index.html')
        else:
            urls.append('https://kgbook.com/list/index_'+ str(index+1) +'.html')
    return urls
    
# 获取当前页面数据
def get_books_in_page(pageUrl, num):
    print('开始分析page', str(num+1), '>>>>>>>')
    s = session.get(pageUrl, headers=headers)
    books = s.html.find('div.channel-item a')
    #a_list = books.find('a')
    return books
    
# 根据一个bookdetailUrl下载书籍
def download_bookfile(pageNum, bookDetailUrl, bookname):
    print('开始解析下载', bookname, '当前页数', str(pageNum+1), '->->->')
    s = session.get(bookDetailUrl, headers=headers)
    btns = s.html.find('a.button')
    for btn in btns:
        bookurl = ''.join(btn.absolute_links)
        # 如果是下载链接
        if 'GetDown' in bookurl:
            realUrl, extendName = get_real_url(bookurl)
            bkname = bookname + '.' + extendName
            # 判断是不是已经存在了这本书
            if is_exist_book(bkname):
                print('\033[1;31m %s 已存在，跳过下载。 \033[0m' % bkname)
                continue
            downloadfile(realUrl, bkname)
    print(bookname, '下载完毕')
    print('-----------------------------')

# 下载文件
def downloadfile(url, filename):
    print('开始下载文件')
    try:
        # r = requests.get(url)
        path = "books/"+filename
        r = requests.get(url, headers=headers)
        with open(path,"wb") as f:
            f.write(r.content)
        f.close()
    except Exception:
        print('\033[1;31m 下载失败 \033[0m')
    else:
        print("\033[1;32m 下载", filename, "完成\033[0m")
    
# 获取重定向真实url
def get_real_url(url):
    print('开始获取真实下载地址')
    # http_headers = { 'Accept': '*/*','Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'}
    rs = requests.get(url,headers=headers)
    print(rs.url)
    # 获取文件的拓展名
    words = rs.url.split('.')
    extendName = words[len(words) - 1]
    print('这是一个', extendName, '文件')
    return rs.url, extendName

# 目录下是否存在这本书
def is_exist_book(bookname):
    if os.path.exists('books/'+bookname):
        return True
    else:
        return False


In [9]:
# 全部最新更新书籍
url = get_last_page_url()
num = get_total_pages(url)
urls = get_total_page_urls(num)
# 遍历page
for index in range(len(urls)):
    #调整起始页
    if index < firstPage:
        print('跳过第%s页' % str(index+1))
        continue
    onepage = urls[index]
    bookset = get_books_in_page(onepage, index)
    # 遍历单页书籍
    for book in bookset:
        # 属性集合
        attrs = book.attrs
        # 如果属性里面没有title字段，说明不是一个书籍详情的链接
        if not 'title' in attrs.keys():
            continue
        title = book.text
        bookDetailUrl = ''.join(book.absolute_links)
        download_bookfile(index, bookDetailUrl, title)

一共81页
跳过第1页
跳过第2页
跳过第3页
跳过第4页
跳过第5页
跳过第6页
跳过第7页
跳过第8页
跳过第9页
跳过第10页
跳过第11页
跳过第12页
跳过第13页
跳过第14页
跳过第15页
跳过第16页
跳过第17页
跳过第18页
跳过第19页
跳过第20页
跳过第21页
跳过第22页
跳过第23页
开始分析page 24 >>>>>>>
开始解析下载 地藏菩萨本愿经白话文及注释 当前页数 24 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/5a435f62a54a8bd92f267efcb845b6f3.pdf
这是一个 pdf 文件
 地藏菩萨本愿经白话文及注释.pdf 已存在，跳过下载。 
地藏菩萨本愿经白话文及注释 下载完毕
-----------------------------
开始解析下载 《读者》2007年合订本 当前页数 24 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/24570f3bd73353e5bf6536b8b0404b66.pdf
这是一个 pdf 文件
 《读者》2007年合订本.pdf 已存在，跳过下载。 
《读者》2007年合订本 下载完毕
-----------------------------
开始解析下载 《读者》2006年合订本 当前页数 24 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/1e599049b198211c82f98f3c1f1372cb.pdf
这是一个 pdf 文件
 《读者》2006年合订本.pdf 已存在，跳过下载。 
《读者》2006年合订本 下载完毕
-----------------------------
开始解析下载 《读者》2005年合订本 当前页数 24 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/a821cb9cc8254fecc731bffdcc9a1ca6.pdf
这是一个 pdf 文件
 《读者》2005年合订本.pdf 已存在，跳过下载。 
《读者》2005年合订本 下载完毕
-----------------------------

https://kgbook.com/d/file/201106/1e0d673b3b8d694922477f2564de5e88.pdf
这是一个 pdf 文件
开始下载文件
 下载 那些回不去的年少时光终场.pdf 完成
那些回不去的年少时光终场 下载完毕
-----------------------------
开始解析下载 那些回不去的年少时光 当前页数 26 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/ed2a931bdd4a8a089ca8cef8b9bf3bde.pdf
这是一个 pdf 文件
开始下载文件
 下载 那些回不去的年少时光.pdf 完成
那些回不去的年少时光 下载完毕
-----------------------------
开始解析下载 天才在左 疯子在右 当前页数 26 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/5dd4d9a4fbdd474df9896882fcf38803.pdf
这是一个 pdf 文件
开始下载文件
 下载 天才在左 疯子在右.pdf 完成
天才在左 疯子在右 下载完毕
-----------------------------
开始解析下载 武侠，从牛A到牛C 当前页数 26 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/e9efe55b990a5da4ac0c9c4d0bf14c5e.pdf
这是一个 pdf 文件
开始下载文件
 下载 武侠，从牛A到牛C.pdf 完成
武侠，从牛A到牛C 下载完毕
-----------------------------
开始解析下载 恒河大手印 当前页数 26 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201106/1cc77d405505cdd138f255964c4d847d.pdf
这是一个 pdf 文件
开始下载文件
 下载 恒河大手印.pdf 完成
恒河大手印 下载完毕
-----------------------------
开始解析下载 竹窗二笔 当前页数 26 ->->->
开始获取真实下载地址
https://kgbook.com

KeyboardInterrupt: 